In [1]:
import numpy as np
import os
import sys
import torch
import matplotlib.pyplot as plt
import time
from DeepQuantile import DQR
import numpy.random as rgt
from scipy.stats import norm, t
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from WAQR import WAQR, FullyConnectedNN
from joint import QuantES
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt


# 1.Parametric Model

This is an example of ES regression with $\tau=0.1$, following DGP1 of Chetverikov, Liu and Tsyvinski (2022)
where $Y=\varepsilon+X'\bar{\beta}$ with $\varepsilon\sim N(0,1)$. Will use half to train the quantile model, and feed half to train the second step.

In [ ]:
B = 10
l2_err = np.zeros((B,3))
out_l2_err = np.zeros((B,3))
runtime = np.zeros((B,3))
n = 3000 # number of training samples
m = 5000 # number of test samples
p = 5
tau = 0.1

betabar = np.array([0.3,0.5,0,0,0])
betabar1=np.array([0.5,0.3,0.5,0,0,0])
itc=0.5

In [ ]:
VaR = norm.ppf(tau)
SQ = norm.ppf(1-tau)
def tail_function(x) :
    return (x if x < VaR else 0)

def right_tail_function(x):
    return (x if x >SQ else 0)

CVaR = norm.expect(tail_function)/tau
CSQ = norm.expect(right_tail_function)/tau

In [ ]:
for b in range(B):
    X = np.random.standard_normal(size=(n, p))
    Xbar= np.hstack((np.ones((n, 1)), X))
    # Add a column of ones as the first column of the matrix
    #X = np.hstack((np.ones((n, 1)), X))
    err = rgt.normal(0,1,n)
    Y = err +X.dot(betabar)+itc
    X_test = np.random.standard_normal(size=(m, p))
    # Add a column of ones as the first column of the matrix
    X_test_bar = np.hstack((np.ones((m, 1)), X_test))
    true_test_ES=X_test.dot(betabar) + CVaR +itc
    true_test_SQ=X_test.dot(betabar) + CSQ +itc
    
    waqr1 = WAQR(Xbar,Y,options={'depth' : 3})
    modells1 = waqr1.fit_ls(tau1=tau,intercept=False)
    betahat=modells1.coef_
    l2_err[b,0] = (np.mean((true_test_ES - modells1.predict(X_test_bar))**2))**0.5
    
    waqr2 = WAQR(Xbar,Y,options={'depth' : 3})
    modelnl1, trainloss = waqr2.fit_nl(tau1=tau,hidden_sizes=[48],dropout_rate=0,lr=0.1, batch_size=32, use_lr_decay=True)
    l2_err[b,1] = (np.mean((true_test_ES - modelnl1.predict(X_test_bar))**2))**0.5
    
    init = QuantES(X, Y)
    ## two-step least squares
    m1 = init.twostep_fit(tau=tau, loss='L2',standardize=True)
    l2_err[b,2] = (np.mean((true_test_ES - X_test_bar.dot(m1['coef_e']))**2))**0.5
    
    #m2 = init.joint_fit(tau=tau)
    #l2_err[b,2] = (np.mean((true_test_ES - X_test_bar.dot(m2['coef_e']))**2))**0.5
    

In [ ]:
np.mean(l2_err, axis = 0)

In [ ]:
plt.plot(np.array(trainloss), label='Training loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
np.array(trainloss)

In [ ]:
len(trainloss)

# 2. Nonparametric Model

In [12]:
B = 10
l2_err_np = np.zeros((B,3))

runtime = np.zeros((B,3))
n = 3000 # number of training samples
m = 5000 # number of test samples
p = 4
tau = 0.1



### (1)
$$ y = \sin(4\pi x_1^2) \sin(4\pi x_2) + 2\cos(4\pi x_3) + x_4+\epsilon$$

In [45]:
nonpar_function = lambda x : np.sin(4*np.pi*(x[:,0]))*np.sin(4*np.pi*x[:,1])+2*np.cos(4*np.pi*x[:,2])+x[:,3]

In [46]:
VaR = norm.ppf(tau)
SQ = norm.ppf(1-tau)
def tail_function(x) :
    return (x if x < VaR else 0)

def right_tail_function(x):
    return (x if x >SQ else 0)

CVaR = norm.expect(tail_function)/tau
CSQ = norm.expect(right_tail_function)/tau

In [ ]:
for b in range(B):
    X = np.random.standard_normal(size=(n, p))
    err = rgt.normal(0,1,n)
    Y = err +nonpar_function(X)
    X_test = np.random.standard_normal(size=(m, p))
    X_test_bar = np.hstack((np.ones((m, 1)), X_test))
    # Add a column of ones as the first column of the matrix
    
    true_test_ES=nonpar_function(X_test) + CVaR 
    true_test_SQ=nonpar_function(X_test) + CSQ 
    
    waqr1 = WAQR(X,Y,options={'depth' : 3})
    modells1 = waqr1.fit_ls(tau1=tau,weighting='es' ,intercept=True)
    l2_err_np[b,0] = (np.mean((true_test_ES - modells1.predict(X_test))**2))**0.5
    
    waqr2 = WAQR(X,Y,options={'depth' : 3})
    modelnl1,train_losses = waqr2.fit_nl(tau1=tau,weighting='es',hidden_sizes=[200,200,200],num_epochs=200,loss_function='Huber',dropout_rate=0,use_lr_decay=True,lr=0.1,batch_size=128, step_size=1,gamma=0.99)
    l2_err_np[b,1] = (np.mean((true_test_ES - modelnl1.predict(X_test))**2))**0.5
    
    init = QuantES(X, Y)
    ## two-step least squares
    m1 = init.twostep_fit(tau=tau, loss='L2',standardize=True)
    l2_err_np[b,2] = (np.mean((true_test_ES - X_test_bar.dot(m1['coef_e']))**2))**0.5
    
    #m2 = init.joint_fit(tau=tau)
    #l2_err[b,2] = (np.mean((true_test_ES - X_test_bar.dot(m2['coef_e']))**2))**0.5
    

In [ ]:
np.mean(l2_err_np, axis = 0)

In [ ]:
plt.plot(np.array(train_losses), label='Training loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
train_losses

In [ ]:
len(train_losses)